In [ ]:
import kagglehub  # pip install kagglehub

# Downloading dataset from Kaggle
# https://www.kaggle.com/datasets/phucthaiv02/butterfly-image-classification
path = kagglehub.dataset_download("phucthaiv02/butterfly-image-classification")

print("Path to dataset files:", path)

In [ ]:
# import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil

In [ ]:
train_csv_file = f"{path}/Training_set.csv"

In [ ]:
# Load training data
train_data = pd.read_csv(train_csv_file)

classes = train_data["label"].unique()

# Shuffle and split %20 test %80 train
train_data = train_data.sample(frac=0.8).reset_index(drop=True)
val_data = train_data.sample(frac=0.2).reset_index(drop=True)
len(classes), len(train_data), len(val_data)

In [ ]:
# Create folder for each class
for class_ in classes:
    os.makedirs(f"./butterfly_dataset/train/{class_}", exist_ok=True)

for class_ in classes:
    os.makedirs(f"./butterfly_dataset/test/{class_}", exist_ok=True)

In [ ]:
for index, row in train_data.iterrows():
    class_ = row["label"]
    filename = row["filename"]
    shutil.copy(
        f"{path}/train/{filename}", f"./butterfly_dataset/train/{class_}/{filename}"
    )

In [ ]:
for index, row in val_data.iterrows():
    class_ = row["label"]
    filename = row["filename"]
    shutil.copy(
        f"{path}/train/{filename}", f"./butterfly_dataset/test/{class_}/{filename}"
    )

In [ ]:
train_dir = "./butterfly_dataset/train/"
val_dir = "./butterfly_dataset/test/"

In [ ]:
# Image transformations
data_transforms = {
    "train": transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
    "val": transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
}

In [ ]:
# Load datasets
image_datasets = {
    "train": datasets.ImageFolder(train_dir, data_transforms["train"]),
    "val": datasets.ImageFolder(val_dir, data_transforms["val"]),
}

In [ ]:
# Data loaders
dataloaders = {
    "train": torch.utils.data.DataLoader(
        image_datasets["train"], batch_size=32, shuffle=True
    ),
    "val": torch.utils.data.DataLoader(
        image_datasets["val"], batch_size=32, shuffle=False
    ),
}

In [ ]:
# Class names
class_names = image_datasets["train"].classes
len(class_names)

In [ ]:
# Initializing the device GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Load ResNet18 model
model = models.resnet18(weights=None)

In [ ]:
# Freeze all layers
for param in model.parameters():
    param.requires_grad = True

In [ ]:
# Modify the classifier
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 75),  # Multi class classification
)

model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    best_model_wts = model.state_dict()
    best_acc = 0.0
    history = {"train": [], "val": []}

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print("-" * 10)

        for phase in ["train", "val"]:
            if phase == "train":
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                # Track loss and accuracy
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
            history[phase].append(epoch_acc)

            # Save best model
            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    print(f"Best val Acc: {best_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model, history

In [ ]:
model, history = train_model(model, dataloaders, criterion, optimizer, num_epochs=10)

In [ ]:
# Plot the training and validation accuracy:

train_acc = [h.item() for h in history["train"]]
val_acc = [h.item() for h in history["val"]]

plt.plot(train_acc, label="Train Accuracy")
plt.plot(val_acc, label="Validation Accuracy")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Training and Validation Accuracy")
plt.show()